In [1]:
import pandas as pd
import numpy as np

In [2]:
target = pd.read_csv('STOP1_targets.csv')

In [3]:
data = pd.read_csv('transcriptome_cpm_all_expressed_genes.csv')

In [4]:
data.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,...,stop1lp6,stop1lp6_2,stop1hp5,stop1hp5_2,stop1lp5,stop1lp5_2,stop1al,stop1al_2,s1fe,s1fe_2
0,AT1G01020,18.512358,17.602610,17.799929,20.364657,18.992583,21.925784,18.287405,21.480752,19.010360,...,20.432136,22.192620,21.396058,22.064116,18.238741,21.516514,21.571426,17.084388,16.595189,21.068461
1,AT1G01030,0.815972,1.068366,0.684613,0.452548,0.542645,0.403048,0.564136,0.290936,0.222995,...,1.104440,0.407544,1.086965,0.735471,1.053156,0.507901,0.596656,0.533887,1.132480,1.344795
2,AT1G01040,36.565732,34.085979,31.915991,53.513793,37.062670,34.783000,43.438463,49.216621,42.090387,...,35.392274,27.638890,31.808043,40.871148,38.152958,37.584640,43.877199,38.973760,23.651412,27.456240
3,AT1G01050,103.577409,109.889125,103.213510,96.223004,100.877751,106.001495,97.595507,100.954685,89.755659,...,109.439940,120.670058,98.685025,97.397312,105.698533,111.045526,86.285705,85.373404,104.536626,106.911235
4,AT1G01060,1.937933,1.780611,2.934054,2.884993,3.147342,3.667732,2.444589,2.133528,1.337973,...,2.510090,2.111819,1.544635,2.364012,1.148897,3.601477,1.331003,1.116309,5.444616,3.361989


In [5]:
data.columns

Index(['atID', 'wthp6', 'wthp6_2', 'wtlp6', 'wtlp6_2', 'wthp5', 'wthp5_2',
       'wtlp5', 'wtlp5_2', 'wtal', 'wtal_2', 'wtfe', 'wtfe_2', 'stop1hp6',
       'stop1hp6_2', 'stop1lp6', 'stop1lp6_2', 'stop1hp5', 'stop1hp5_2',
       'stop1lp5', 'stop1lp5_2', 'stop1al', 'stop1al_2', 's1fe', 's1fe_2'],
      dtype='object')

In [6]:
wild_type = 'wthp6'
treatments = ['stop1hp6', 'stop1lp6', 'stop1hp5', 'stop1lp5', 'stop1al', 's1fe']

In [7]:
target.head()

,target.at_id
0,AT1G01020
1,AT1G01180
2,AT1G01190
3,AT1G01490
4,AT1G01530


In [8]:
from scipy import stats
import matplotlib.pyplot as plt

In [9]:
#!pip install mne

In [10]:
from mne.stats import fdr_correction

In [11]:
def analyze_treatment(df, treatment_name, alpha):
    wt_data = df[['wthp6', 'wthp6' ]].values
    T, pval = stats.ttest_ind(a=wt_data, b=df[[treatment_name, treatment_name+"_2"]].values, axis=1)
    reject_fdr, pval_fdr = fdr_correction(pval, alpha=alpha, method='indep')
    return reject_fdr

In [12]:
def process_significant(df, alpha):
    for treatment_name in treatments:
        df[treatment_name+"_sig"] = analyze_treatment(df, treatment_name, alpha)

In [13]:
filtered_data = data[data.atID.isin(target['target.at_id'].values)]

In [14]:
alpha = 0.05
# process_significant(data, alpha)
process_significant(filtered_data, alpha)

/Users/vanpham/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/vanpham/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/vanpham/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/Users/vanpham/opt/anaconda3/lib/python3.7/site-packages/mne/stats/multi_comp.py:65: RuntimeWarning: invalid value encountered in less
  reject = pvals_sorted < (ecdffactor * alpha)
/Users/vanpham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [15]:
def significant_value_counts(df):
    for treatment_name in treatments:
        print(df[treatment_name + '_sig'].value_counts())

In [16]:
# significant_value_counts(data)
significant_value_counts(filtered_data)

False    814
True      82
Name: stop1hp6_sig, dtype: int64
False    786
True     110
Name: stop1lp6_sig, dtype: int64
False    816
True      80
Name: stop1hp5_sig, dtype: int64
False    805
True      91
Name: stop1lp5_sig, dtype: int64
False    521
True     375
Name: stop1al_sig, dtype: int64
False    594
True     302
Name: s1fe_sig, dtype: int64


In [17]:
def up_down_process(df, treatment_name):
    wt_mean = np.mean(df[[wild_type, wild_type + "_2"]].values, axis = 1)
    tm_mean = np.mean(df[[treatment_name, treatment_name + "_2"]].values, axis = 1)
    df_mean = wt_mean - tm_mean
    tm_sig = df[treatment_name+"_sig"].values
    return [0 if sig == False else 1 if df_mean[i] < 0 else -1 for i, sig in enumerate(tm_sig)]

In [18]:
def process_up_down_data(df):
    for treatment_name in treatments:
        df[treatment_name+"_up_down"] = up_down_process(df, treatment_name)

In [19]:
# process_up_down_data(data)
process_up_down_data(filtered_data)

/Users/vanpham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
def count_up_down_values(df):
    for treatment_name in treatments:
        print(df[treatment_name + '_up_down'].value_counts())

In [21]:
# count_up_down_values(data)
count_up_down_values(filtered_data)

 0    814
-1     56
 1     26
Name: stop1hp6_up_down, dtype: int64
 0    786
-1     74
 1     36
Name: stop1lp6_up_down, dtype: int64
 0    816
-1     45
 1     35
Name: stop1hp5_up_down, dtype: int64
 0    805
-1     59
 1     32
Name: stop1lp5_up_down, dtype: int64
 0    521
-1    209
 1    166
Name: stop1al_up_down, dtype: int64
 0    594
-1    204
 1     98
Name: s1fe_up_down, dtype: int64


In [22]:
# def get_up_entries(df):
#     up_down_values = df[[treatment_name + "_up_down" for treatment_name in treatments]].values
#     up_filter = [True if 1 in row and -1 not in row else False for row in up_down_values]
#     return up_filter

def get_up_entries(df):
    up_down_values = df[[treatment_name + "_up_down" for treatment_name in treatments]].values
    up_filter = [True if 1 in row else False for row in up_down_values]
    return up_filter

In [23]:
# def get_down_entries(df):
#     up_down_values = df[[treatment_name + "_up_down" for treatment_name in treatments]].values
#     down_filter = [True if -1 in row and 1 not in row else False for row in up_down_values]
#     return down_filter

def get_down_entries(df):
    up_down_values = df[[treatment_name + "_up_down" for treatment_name in treatments]].values
    down_filter = [True if -1 in row else False for row in up_down_values]
    return down_filter

In [24]:
# up_df = data[get_up_entries(data)]
up_df = filtered_data[get_up_entries(filtered_data)]

In [25]:
len(up_df)

305

In [26]:
# down_df = data[get_down_entries(data)]
down_df = filtered_data[get_down_entries(filtered_data)]

In [27]:
len(down_df)

410

In [28]:
def get_up_and_down_entries(df):
    up_down_values = df[[treatment_name + "_up_down" for treatment_name in treatments]].values
    up_down_filter = [True if -1 in row and 1 in row else False for row in up_down_values]
    return up_down_filter

In [29]:
# up_down_df = data[get_up_and_down_entries(data)]
up_down_df = filtered_data[get_up_and_down_entries(filtered_data)]

In [30]:
len(up_down_df)

80

In [31]:
filtered_up_df = up_df[up_df.atID.isin(target['target.at_id'].values)]
len(filtered_up_df)

305

In [32]:
filtered_up_df.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,...,stop1hp5_sig,stop1lp5_sig,stop1al_sig,s1fe_sig,stop1hp6_up_down,stop1lp6_up_down,stop1hp5_up_down,stop1lp5_up_down,stop1al_up_down,s1fe_up_down
39,AT1G01490,91.847816,113.246848,98.551624,107.310428,127.033251,124.581985,103.612956,118.071402,88.473435,...,False,False,False,True,1,0,0,0,0,-1
57,AT1G01710,30.394946,28.438899,27.514909,31.225807,28.977256,26.722050,27.266567,29.675441,31.776848,...,False,False,False,True,0,0,0,0,0,1
351,AT1G05000,47.938338,52.502582,127.468359,96.845257,70.598146,58.643412,111.604881,114.483195,181.183784,...,False,True,True,True,-1,1,0,1,1,-1
369,AT1G05170,14.891484,14.702758,15.485287,13.067321,16.170828,17.935614,16.689020,14.934699,16.613159,...,False,False,False,True,0,0,0,0,0,1
398,AT1G05430,22.796210,26.454789,23.211630,21.778869,24.364771,22.449746,23.740718,24.875002,21.574808,...,False,True,False,False,1,0,0,1,0,0


In [33]:
filtered_down_df = down_df[down_df.atID.isin(target['target.at_id'].values)]
len(filtered_down_df)

410

In [34]:
filtered_down_df.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,...,stop1hp5_sig,stop1lp5_sig,stop1al_sig,s1fe_sig,stop1hp6_up_down,stop1lp6_up_down,stop1hp5_up_down,stop1lp5_up_down,stop1al_up_down,s1fe_up_down
16,AT1G01190,27.896033,17.653484,21.255593,19.063582,22.736836,11.970511,38.220206,24.099173,34.564291,...,False,False,False,False,0,-1,0,0,0,0
39,AT1G01490,91.847816,113.246848,98.551624,107.310428,127.033251,124.581985,103.612956,118.071402,88.473435,...,False,False,False,True,1,0,0,0,0,-1
66,AT1G01780,33.352844,27.065284,40.163943,25.286116,41.349567,42.481207,34.929413,39.130851,45.323820,...,False,False,True,False,0,0,0,0,-1,0
81,AT1G01960,126.169626,121.386783,119.905019,190.296404,135.932633,132.118973,158.193102,151.674474,153.643849,...,False,False,False,True,0,0,0,0,0,-1
153,AT1G02890,118.876878,121.997278,115.503937,171.289391,135.281459,121.800957,143.572581,134.703226,126.494156,...,False,False,False,True,0,0,0,0,0,-1


In [35]:
filtered_up_down_df = up_down_df[up_down_df.atID.isin(target['target.at_id'].values)]
len(filtered_up_down_df)

80

In [36]:
filtered_up_down_df.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,...,stop1hp5_sig,stop1lp5_sig,stop1al_sig,s1fe_sig,stop1hp6_up_down,stop1lp6_up_down,stop1hp5_up_down,stop1lp5_up_down,stop1al_up_down,s1fe_up_down
39,AT1G01490,91.847816,113.246848,98.551624,107.310428,127.033251,124.581985,103.612956,118.071402,88.473435,...,False,False,False,True,1,0,0,0,0,-1
351,AT1G05000,47.938338,52.502582,127.468359,96.845257,70.598146,58.643412,111.604881,114.483195,181.183784,...,False,True,True,True,-1,1,0,1,1,-1
702,AT1G08340,23.816174,20.095465,27.873516,19.572698,29.194314,32.646848,32.014711,31.566523,25.086985,...,False,False,True,True,0,0,0,0,-1,1
809,AT1G09310,4.640839,1.119241,1.206222,0.791959,0.813968,9.028264,0.846204,1.357700,14.383205,...,True,True,True,False,0,0,1,-1,-1,0
1076,AT1G12110,1112.526428,1169.454312,856.450448,1326.191707,1255.735355,1272.058240,1567.780615,1305.186006,1287.352598,...,True,False,True,True,0,0,1,0,-1,-1


In [37]:
target_diff_expressed = pd.read_csv('Targets_differentially_expressed.csv')

In [38]:
target_diff_expressed.head()

,TARGETS|upregulated,TARGETS|downregulated,TARGETS|up|down
0,AT1G01190,AT1G01180,AT2G27120
1,AT1G05000,AT1G03760,AT2G35710
2,AT1G05170,AT1G04800,AT3G51330
3,AT1G06650,AT1G06420,AT3G62830
4,AT1G08430,AT1G09780,AT4G10550


In [43]:
common_up = filtered_up_df[filtered_up_df.atID.isin(target_diff_expressed['TARGETS|upregulated'].values)]
given_up = 175
print(f'common up {len(common_up)} out of {given_up} given and {len(filtered_up_df)} up found')

common up 101 out of 175 given and 305 up found


In [40]:
common_down = filtered_down_df[filtered_down_df.atID.isin(target_diff_expressed['TARGETS|downregulated'].values)]
given_down = 114
print(f'common down {len(common_down)} out of {given_down} given and {len(filtered_down_df)} down found')

common down 87 out of 114 given and 410 down found


In [44]:
common_up_down = filtered_up_down_df[filtered_up_down_df.atID.isin(target_diff_expressed['TARGETS|up|down'].values)]
given_up_down = 7
print(f'common up down {len(common_up_down)} out of {given_up_down} given and {len(filtered_up_down_df)} up down found')

common up down 0 out of 7 given and 80 up down found
